# Library

In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pointbiserialr

# Import Data

In [2]:
df = pd.read_excel('DATA RUMAH.xlsx')
df

,NO,NAMA RUMAH,HARGA,LB,LT,KT,KM,GRS
0,1,"Rumah Murah Hook Tebet Timur, Tebet, Jakarta S...",3800000000,220,220,3,3,0
1,2,"Rumah Modern di Tebet dekat Stasiun, Tebet, Ja...",4600000000,180,137,4,3,2
2,3,"Rumah Mewah 2 Lantai Hanya 3 Menit Ke Tebet, T...",3000000000,267,250,4,4,4
3,4,"Rumah Baru Tebet, Tebet, Jakarta Selatan",430000000,40,25,2,2,0
4,5,"Rumah Bagus Tebet komp Gudang Peluru lt 350m, ...",9000000000,400,355,6,5,3
...,...,...,...,...,...,...,...,...
1005,1006,Rumah Strategis Akses Jalan 2mobil Di Menteng ...,9000000000,450,550,10,10,3
1006,1007,Tebet Rumah Siap Huni Jln 2 Mbl Nyaman,4000000000,160,140,4,3,2
1007,1008,"Di Kebun Baru Rumah Terawat, Area Strategis",4000000000,139,230,4,4,1
1008,1009,Dijual Cepat Rumah Komp Depkeu Dr Soepomo Tebe...,19000000000,360,606,7,4,0


# Penanganan Kolom

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010 entries, 0 to 1009
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NO          1010 non-null   int64 
 1   NAMA RUMAH  1010 non-null   object
 2   HARGA       1010 non-null   int64 
 3   LB          1010 non-null   int64 
 4   LT          1010 non-null   int64 
 5   KT          1010 non-null   int64 
 6   KM          1010 non-null   int64 
 7   GRS         1010 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 63.2+ KB


In [4]:
df.shape

(1010, 8)

## Mengubah Header Kolom

In [5]:
# rename detail header kolom
df.columns = ['no', 'nama','harga', 'luas_tanah', 'luas_bangunan', 'kamar_tidur', 'kamar_mandi', 'garasi']
df.head()

,no,nama,harga,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi
0,1,"Rumah Murah Hook Tebet Timur, Tebet, Jakarta S...",3800000000,220,220,3,3,0
1,2,"Rumah Modern di Tebet dekat Stasiun, Tebet, Ja...",4600000000,180,137,4,3,2
2,3,"Rumah Mewah 2 Lantai Hanya 3 Menit Ke Tebet, T...",3000000000,267,250,4,4,4
3,4,"Rumah Baru Tebet, Tebet, Jakarta Selatan",430000000,40,25,2,2,0
4,5,"Rumah Bagus Tebet komp Gudang Peluru lt 350m, ...",9000000000,400,355,6,5,3


## Kolom Nama

In [6]:
df['nama'] = df['nama'].str.lower()
df.head()

,no,nama,harga,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi
0,1,"rumah murah hook tebet timur, tebet, jakarta s...",3800000000,220,220,3,3,0
1,2,"rumah modern di tebet dekat stasiun, tebet, ja...",4600000000,180,137,4,3,2
2,3,"rumah mewah 2 lantai hanya 3 menit ke tebet, t...",3000000000,267,250,4,4,4
3,4,"rumah baru tebet, tebet, jakarta selatan",430000000,40,25,2,2,0
4,5,"rumah bagus tebet komp gudang peluru lt 350m, ...",9000000000,400,355,6,5,3


# Penanganan Missing Value

In [7]:
df.isna().sum()

no               0
nama             0
harga            0
luas_tanah       0
luas_bangunan    0
kamar_tidur      0
kamar_mandi      0
garasi           0
dtype: int64

# Feature Engineering

## Kolom Nama

In [8]:
q = df['harga'].quantile([.25, .5, .75])
q

0.25    3.262500e+09
0.50    5.000000e+09
0.75    9.000000e+09
Name: harga, dtype: float64

In [9]:
def group_by_title_category(price):
    if(price <= q[0.25]):
        return 'standard'
    elif(price > q[0.25] and price  <= q[0.50]):
        return 'modern'
    elif(price > q[0.50]):
        return 'exclusive'
    else:
        return '-'

df = df.copy()
df['kelas_title'] = df['harga'].apply(group_by_title_category)
df.head()

,no,nama,harga,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,kelas_title
0,1,"rumah murah hook tebet timur, tebet, jakarta s...",3800000000,220,220,3,3,0,modern
1,2,"rumah modern di tebet dekat stasiun, tebet, ja...",4600000000,180,137,4,3,2,modern
2,3,"rumah mewah 2 lantai hanya 3 menit ke tebet, t...",3000000000,267,250,4,4,4,standard
3,4,"rumah baru tebet, tebet, jakarta selatan",430000000,40,25,2,2,0,standard
4,5,"rumah bagus tebet komp gudang peluru lt 350m, ...",9000000000,400,355,6,5,3,exclusive


In [10]:
df = df.drop('nama', axis=1)
df.head()

,no,harga,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,kelas_title
0,1,3800000000,220,220,3,3,0,modern
1,2,4600000000,180,137,4,3,2,modern
2,3,3000000000,267,250,4,4,4,standard
3,4,430000000,40,25,2,2,0,standard
4,5,9000000000,400,355,6,5,3,exclusive


## Kolom Harga

In [11]:
df['harga(jt)'] = df['harga']//1000000
df.head()

,no,harga,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,kelas_title,harga(jt)
0,1,3800000000,220,220,3,3,0,modern,3800
1,2,4600000000,180,137,4,3,2,modern,4600
2,3,3000000000,267,250,4,4,4,standard,3000
3,4,430000000,40,25,2,2,0,standard,430
4,5,9000000000,400,355,6,5,3,exclusive,9000


In [12]:
q = df['harga(jt)'].quantile([.25, .5, .75])
q

0.25    3262.5
0.50    5000.0
0.75    9000.0
Name: harga(jt), dtype: float64

In [13]:
def group_by_price_category(price):
    if(price <= q[0.25]):
        return 'harga rendah'
    elif(price > q[0.25] and price  <= q[0.50]):
        return 'harga menengah'
    elif(price > q[0.50]):
        return 'harga tinggi'
    else:
        return '-'

df = df.copy()
df['kelas_harga'] = df['harga(jt)'].apply(group_by_price_category)
df.head()

,no,harga,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,kelas_title,harga(jt),kelas_harga
0,1,3800000000,220,220,3,3,0,modern,3800,harga menengah
1,2,4600000000,180,137,4,3,2,modern,4600,harga menengah
2,3,3000000000,267,250,4,4,4,standard,3000,harga rendah
3,4,430000000,40,25,2,2,0,standard,430,harga rendah
4,5,9000000000,400,355,6,5,3,exclusive,9000,harga tinggi


In [14]:
df = df.drop('harga', axis=1)
df = df[['no', 'luas_tanah', 'luas_bangunan', 'kamar_tidur',
       'kamar_mandi', 'garasi', 'harga(jt)','kelas_title', 'kelas_harga']]
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga
0,1,220,220,3,3,0,3800,modern,harga menengah
1,2,180,137,4,3,2,4600,modern,harga menengah
2,3,267,250,4,4,4,3000,standard,harga rendah
3,4,40,25,2,2,0,430,standard,harga rendah
4,5,400,355,6,5,3,9000,exclusive,harga tinggi


## Kolom Luas Tanah

In [15]:
q = df['luas_tanah'].quantile([.25, .5, .75])
q

0.25    150.0
0.50    216.5
0.75    350.0
Name: luas_tanah, dtype: float64

In [16]:
def group_by_meter_category(price):
    if(price <= q[0.25]):
        return 'kurang luas'
    elif(price > q[0.25] and price  <= q[0.50]):
        return 'sedang'
    elif(price > q[0.50]):
        return 'luas'
    else:
        return '-'

df = df.copy()
df['kelas_luas'] = df['luas_tanah'].apply(group_by_meter_category)
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas
0,1,220,220,3,3,0,3800,modern,harga menengah,luas
1,2,180,137,4,3,2,4600,modern,harga menengah,sedang
2,3,267,250,4,4,4,3000,standard,harga rendah,luas
3,4,40,25,2,2,0,430,standard,harga rendah,kurang luas
4,5,400,355,6,5,3,9000,exclusive,harga tinggi,luas


# Kolom Garasi

In [17]:
df['ada_garasi'] = df['garasi'].apply(lambda x: 'tidak' if x == 0 else 'ada')
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,ada_garasi
0,1,220,220,3,3,0,3800,modern,harga menengah,luas,tidak
1,2,180,137,4,3,2,4600,modern,harga menengah,sedang,ada
2,3,267,250,4,4,4,3000,standard,harga rendah,luas,ada
3,4,40,25,2,2,0,430,standard,harga rendah,kurang luas,tidak
4,5,400,355,6,5,3,9000,exclusive,harga tinggi,luas,ada


# Encoding

## Kolom Kelas Title

In [18]:
replace_dict = {'standard': 0, 'modern': 1, 'exclusive':2}

# Replace the values in 'col2' using the replace_dict
df['kelas_title'] = df['kelas_title'].replace(replace_dict)
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,ada_garasi
0,1,220,220,3,3,0,3800,1,harga menengah,luas,tidak
1,2,180,137,4,3,2,4600,1,harga menengah,sedang,ada
2,3,267,250,4,4,4,3000,0,harga rendah,luas,ada
3,4,40,25,2,2,0,430,0,harga rendah,kurang luas,tidak
4,5,400,355,6,5,3,9000,2,harga tinggi,luas,ada


## Kolom Kelas Harga

In [19]:
replace_dict = {'harga rendah': 0, 'harga menengah': 1, 'harga tinggi':2,}

# Replace the values in 'col2' using the replace_dict
df['kelas_harga'] = df['kelas_harga'].replace(replace_dict)
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,ada_garasi
0,1,220,220,3,3,0,3800,1,1,luas,tidak
1,2,180,137,4,3,2,4600,1,1,sedang,ada
2,3,267,250,4,4,4,3000,0,0,luas,ada
3,4,40,25,2,2,0,430,0,0,kurang luas,tidak
4,5,400,355,6,5,3,9000,2,2,luas,ada


## Kolom Kelas Luas

In [20]:
replace_dict = {'kurang luas': 0, 'sedang': 1, 'luas':2,}

# Replace the values in 'col2' using the replace_dict
df['kelas_luas'] = df['kelas_luas'].replace(replace_dict)
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,ada_garasi
0,1,220,220,3,3,0,3800,1,1,2,tidak
1,2,180,137,4,3,2,4600,1,1,1,ada
2,3,267,250,4,4,4,3000,0,0,2,ada
3,4,40,25,2,2,0,430,0,0,0,tidak
4,5,400,355,6,5,3,9000,2,2,2,ada


## Kolom Kelas Ada Garasi

In [21]:
garasi = pd.get_dummies(df['ada_garasi'], prefix='garasi')
df = df.join(garasi)
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,ada_garasi,garasi_ada,garasi_tidak
0,1,220,220,3,3,0,3800,1,1,2,tidak,0,1
1,2,180,137,4,3,2,4600,1,1,1,ada,1,0
2,3,267,250,4,4,4,3000,0,0,2,ada,1,0
3,4,40,25,2,2,0,430,0,0,0,tidak,0,1
4,5,400,355,6,5,3,9000,2,2,2,ada,1,0


# Feature Selection

In [22]:
df.head()

,no,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,ada_garasi,garasi_ada,garasi_tidak
0,1,220,220,3,3,0,3800,1,1,2,tidak,0,1
1,2,180,137,4,3,2,4600,1,1,1,ada,1,0
2,3,267,250,4,4,4,3000,0,0,2,ada,1,0
3,4,40,25,2,2,0,430,0,0,0,tidak,0,1
4,5,400,355,6,5,3,9000,2,2,2,ada,1,0


In [23]:
df = df.drop(['no','ada_garasi','garasi_tidak'], axis=1)
df.head()

,luas_tanah,luas_bangunan,kamar_tidur,kamar_mandi,garasi,harga(jt),kelas_title,kelas_harga,kelas_luas,garasi_ada
0,220,220,3,3,0,3800,1,1,2,0
1,180,137,4,3,2,4600,1,1,1,1
2,267,250,4,4,4,3000,0,0,2,1
3,40,25,2,2,0,430,0,0,0,0
4,400,355,6,5,3,9000,2,2,2,1


In [25]:
for column in df.columns:
    if column != 'harga(jt)':  # Exclude 'harga(jt)' column from correlation calculations
        correlation, p_value = pointbiserialr(df[column], df['harga(jt)'])
        print(f"Korelasi {column} dan harga(jt): {correlation}, p-value: {p_value}")

Korelasi luas_tanah dan harga(jt): 0.747082790606928, p-value: 5.668845250375793e-181
Korelasi luas_bangunan dan harga(jt): 0.8096174126682212, p-value: 1.7678608618539863e-235
Korelasi kamar_tidur dan harga(jt): 0.3233201323242367, p-value: 5.237707151037716e-26
Korelasi kamar_mandi dan harga(jt): 0.40477015842374975, p-value: 4.215708264989426e-41
Korelasi garasi dan harga(jt): 0.48489183005934927, p-value: 1.1068031909117222e-60
Korelasi kelas_title dan harga(jt): 0.586021706336339, p-value: 3.4983670924742375e-94
Korelasi kelas_harga dan harga(jt): 0.586021706336339, p-value: 3.4983670924742375e-94
Korelasi kelas_luas dan harga(jt): 0.49760421668671734, p-value: 2.6866629798125645e-64
Korelasi garasi_ada dan harga(jt): 0.07720602301126032, p-value: 0.014117039344394127


# Modeling

In [26]:
X = df.drop('harga(jt)', axis=1)
y = df['harga(jt)']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBRegressor

In [28]:
from xgboost import XGBRegressor

## Split Data

In [29]:
# training
print('Training')
xgboost = XGBRegressor(random_state=42)
xgboost.fit(x_train, y_train)
y_pred = xgboost.predict(x_train)
print('r2_score : ', r2_score(y_train, y_pred))
print('MAE : ', mean_absolute_error(y_train, y_pred))
print('MSE : ', mean_squared_error(y_train, y_pred))

Training
r2_score :  0.9986310925767626
MAE :  150.97832938468102
MSE :  76120.03047553678


In [30]:
# testing
print('Testing')
y_pred_test = xgboost.predict(x_test)
print('r2_score : ', r2_score(y_test, y_pred_test))
print('MAE : ', mean_absolute_error(y_test, y_pred_test))
print('MSE : ', mean_squared_error(y_test, y_pred_test))

Testing
r2_score :  0.783258291859144
MAE :  1424.9775435948136
MSE :  10118103.495290278


In [31]:
prediksi = pd.DataFrame(y_test.values, columns = ['y_actual'])
prediksi['y_predict'] = y_pred_test
prediksi

,y_actual,y_predict
0,8900,6474.796387
1,6500,5904.695801
2,6500,6886.754395
3,37000,33963.769531
4,18500,21720.406250
...,...,...
197,4595,3802.802002
198,18500,20468.083984
199,2100,2630.685059
200,3300,3622.243164
